In [2]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m1.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=0.5,  # p=q=1时，退化为DeepWalk
    q=1.0,
    sparse=False,  # 该数据集为密集数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

tensor(899, device='cuda:1')
torch.Size([2, 21026])
Epoch: 001, Loss: 8.4387,AUC: 0.5571,AP: 0.5289
Epoch: 002, Loss: 7.4364,AUC: 0.6068,AP: 0.5561
Epoch: 003, Loss: 6.6292,AUC: 0.6467,AP: 0.5827
Epoch: 004, Loss: 5.9091,AUC: 0.6784,AP: 0.6073
Epoch: 005, Loss: 5.5818,AUC: 0.7061,AP: 0.6314
Epoch: 006, Loss: 4.9642,AUC: 0.7309,AP: 0.6576
Epoch: 007, Loss: 4.5740,AUC: 0.7521,AP: 0.6816
Epoch: 008, Loss: 4.3000,AUC: 0.7700,AP: 0.7032
Epoch: 009, Loss: 3.9761,AUC: 0.7844,AP: 0.7208
Epoch: 010, Loss: 3.6946,AUC: 0.7954,AP: 0.7354
Epoch: 011, Loss: 3.4465,AUC: 0.8031,AP: 0.7457
Epoch: 012, Loss: 3.2752,AUC: 0.8082,AP: 0.7521
Epoch: 013, Loss: 3.0655,AUC: 0.8128,AP: 0.7591
Epoch: 014, Loss: 2.9085,AUC: 0.8179,AP: 0.7682
Epoch: 015, Loss: 2.6947,AUC: 0.8224,AP: 0.7758
Epoch: 016, Loss: 2.5518,AUC: 0.8262,AP: 0.7816
Epoch: 017, Loss: 2.4428,AUC: 0.8299,AP: 0.7868
Epoch: 018, Loss: 2.3172,AUC: 0.8334,AP: 0.7927
Epoch: 019, Loss: 2.1957,AUC: 0.8368,AP: 0.7972
Epoch: 020, Loss: 2.1144,AUC: 0.8403

In [3]:
z1 = model()
z1.shape
# 节点数实际是899，这里是由于图节点编号从1开始，而列表从0开始，所以对于索引为0的特征向量应该不会被用到

torch.Size([900, 128])

In [4]:
z1[0]

tensor([-0.2721,  0.0995, -0.1786, -0.0347, -0.1345,  0.3340,  0.1330,  0.2713,
         0.2525, -0.0428,  0.0860, -0.1148, -0.1609,  0.0208, -0.1687, -0.3123,
         0.2585, -0.3620,  0.0265, -0.0585,  0.2145,  0.0852, -0.0614,  0.1562,
        -0.0185,  0.1677,  0.0349, -0.3967, -0.0485, -0.0065, -0.0039,  0.0877,
        -0.5729, -0.4422, -0.1665, -0.1857, -0.1774, -0.1686,  0.2383, -0.1294,
         0.1029, -0.0528, -0.2724, -0.1697,  0.1196,  0.2347,  0.1430,  0.1387,
        -0.1743, -0.2277,  0.1404,  0.2274, -0.0846,  0.0153,  0.0104,  0.1668,
         0.0357,  0.3507, -0.0098, -0.1247, -0.0062, -0.0745, -0.0884, -0.0454,
        -0.0654, -0.1711,  0.0819,  0.2387,  0.1617,  0.0998,  0.1661,  0.1250,
         0.1807, -0.1691,  0.2505,  0.0108,  0.3105,  0.0381, -0.0101,  0.5680,
         0.0203, -0.0370, -0.3498, -0.5207,  0.0874, -0.0792,  0.2913,  0.0314,
         0.3772, -0.1924, -0.0456,  0.0973,  0.0257,  0.1601,  0.0082, -0.3676,
        -0.1476,  0.4315, -0.2282, -0.19

In [5]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m2.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=0.5,  # p=q=1时，退化为DeepWalk
    q=1.0,
    sparse=False,  # 该数据集为密集数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

tensor(897, device='cuda:1')
torch.Size([2, 4762])
Epoch: 001, Loss: 7.4532,AUC: 0.6031,AP: 0.5572
Epoch: 002, Loss: 6.2507,AUC: 0.6653,AP: 0.5956
Epoch: 003, Loss: 5.5973,AUC: 0.7076,AP: 0.6283
Epoch: 004, Loss: 5.3504,AUC: 0.7382,AP: 0.6556
Epoch: 005, Loss: 4.9243,AUC: 0.7594,AP: 0.6737
Epoch: 006, Loss: 4.6742,AUC: 0.7752,AP: 0.6903
Epoch: 007, Loss: 4.2765,AUC: 0.7859,AP: 0.7047
Epoch: 008, Loss: 4.0335,AUC: 0.7958,AP: 0.7180
Epoch: 009, Loss: 3.7483,AUC: 0.8051,AP: 0.7322
Epoch: 010, Loss: 3.5175,AUC: 0.8146,AP: 0.7444
Epoch: 011, Loss: 3.2648,AUC: 0.8209,AP: 0.7530
Epoch: 012, Loss: 3.1859,AUC: 0.8272,AP: 0.7625
Epoch: 013, Loss: 2.9542,AUC: 0.8338,AP: 0.7720
Epoch: 014, Loss: 2.8077,AUC: 0.8396,AP: 0.7796
Epoch: 015, Loss: 2.6489,AUC: 0.8434,AP: 0.7865
Epoch: 016, Loss: 2.5250,AUC: 0.8481,AP: 0.7920
Epoch: 017, Loss: 2.4336,AUC: 0.8523,AP: 0.7981
Epoch: 018, Loss: 2.3137,AUC: 0.8572,AP: 0.8042
Epoch: 019, Loss: 2.2464,AUC: 0.8619,AP: 0.8110
Epoch: 020, Loss: 2.1378,AUC: 0.8667,

In [6]:
z2 = model()
z2.shape

torch.Size([898, 128])

In [7]:
zero_2 = torch.zeros((2, 128)).to(device)
z2 = torch.cat([z2, zero_2], 0)
z2.shape

torch.Size([900, 128])

In [8]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m3.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=0.5,  # p=q=1时，退化为DeepWalk
    q=1.0,
    sparse=False,  # 该数据集为密集数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

tensor(897, device='cuda:1')
torch.Size([2, 3220])
Epoch: 001, Loss: 6.7476,AUC: 0.5529,AP: 0.5215
Epoch: 002, Loss: 5.7288,AUC: 0.6123,AP: 0.5529
Epoch: 003, Loss: 5.2171,AUC: 0.6597,AP: 0.5801
Epoch: 004, Loss: 4.8983,AUC: 0.6985,AP: 0.6097
Epoch: 005, Loss: 4.4788,AUC: 0.7264,AP: 0.6338
Epoch: 006, Loss: 4.2856,AUC: 0.7464,AP: 0.6522
Epoch: 007, Loss: 3.9368,AUC: 0.7595,AP: 0.6689
Epoch: 008, Loss: 3.7693,AUC: 0.7711,AP: 0.6836
Epoch: 009, Loss: 3.4885,AUC: 0.7800,AP: 0.6935
Epoch: 010, Loss: 3.2874,AUC: 0.7864,AP: 0.7038
Epoch: 011, Loss: 3.0603,AUC: 0.7916,AP: 0.7118
Epoch: 012, Loss: 2.9724,AUC: 0.7944,AP: 0.7174
Epoch: 013, Loss: 2.7783,AUC: 0.8022,AP: 0.7301
Epoch: 014, Loss: 2.6616,AUC: 0.8094,AP: 0.7403
Epoch: 015, Loss: 2.4965,AUC: 0.8142,AP: 0.7492
Epoch: 016, Loss: 2.3796,AUC: 0.8187,AP: 0.7556
Epoch: 017, Loss: 2.2688,AUC: 0.8214,AP: 0.7615
Epoch: 018, Loss: 2.1738,AUC: 0.8248,AP: 0.7687
Epoch: 019, Loss: 2.1052,AUC: 0.8302,AP: 0.7768
Epoch: 020, Loss: 2.0102,AUC: 0.8371,

In [9]:
z3 = model()
z3.shape

torch.Size([898, 128])

In [10]:
zero_2 = torch.zeros((2, 128)).to(device)
z3 = torch.cat([z3, zero_2], 0)
z3.shape

torch.Size([900, 128])

In [11]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m4.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=0.5,  # p=q=1时，退化为DeepWalk
    q=1.0,
    sparse=False,  # 该数据集为密集数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

tensor(897, device='cuda:1')
torch.Size([2, 3052])
Epoch: 001, Loss: 6.1907,AUC: 0.6365,AP: 0.5775
Epoch: 002, Loss: 5.4508,AUC: 0.6958,AP: 0.6120
Epoch: 003, Loss: 4.9338,AUC: 0.7324,AP: 0.6397
Epoch: 004, Loss: 4.7292,AUC: 0.7558,AP: 0.6606
Epoch: 005, Loss: 4.3182,AUC: 0.7802,AP: 0.6844
Epoch: 006, Loss: 4.0732,AUC: 0.7961,AP: 0.7016
Epoch: 007, Loss: 3.7962,AUC: 0.8058,AP: 0.7147
Epoch: 008, Loss: 3.6290,AUC: 0.8114,AP: 0.7211
Epoch: 009, Loss: 3.3760,AUC: 0.8209,AP: 0.7356
Epoch: 010, Loss: 3.1670,AUC: 0.8257,AP: 0.7411
Epoch: 011, Loss: 2.9480,AUC: 0.8303,AP: 0.7487
Epoch: 012, Loss: 2.8597,AUC: 0.8346,AP: 0.7542
Epoch: 013, Loss: 2.6334,AUC: 0.8413,AP: 0.7708
Epoch: 014, Loss: 2.5459,AUC: 0.8459,AP: 0.7763
Epoch: 015, Loss: 2.4017,AUC: 0.8505,AP: 0.7854
Epoch: 016, Loss: 2.2838,AUC: 0.8540,AP: 0.7938
Epoch: 017, Loss: 2.1770,AUC: 0.8575,AP: 0.8020
Epoch: 018, Loss: 2.0756,AUC: 0.8615,AP: 0.8115
Epoch: 019, Loss: 2.0033,AUC: 0.8664,AP: 0.8204
Epoch: 020, Loss: 1.9084,AUC: 0.8719,

In [12]:
z4 = model()
z4.shape

torch.Size([898, 128])

In [13]:
zero_2 = torch.zeros((2, 128)).to(device)
z4 = torch.cat([z4, zero_2], 0)
z4.shape

torch.Size([900, 128])

In [14]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m5.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=0.5,  # p=q=1时，退化为DeepWalk
    q=1.0,
    sparse=False,  # 该数据集为密集数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 151):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

tensor(897, device='cuda:1')
torch.Size([2, 1952])
Epoch: 001, Loss: 6.3228,AUC: 0.5826,AP: 0.5433
Epoch: 002, Loss: 5.5022,AUC: 0.6529,AP: 0.5834
Epoch: 003, Loss: 4.9748,AUC: 0.7054,AP: 0.6203
Epoch: 004, Loss: 4.7265,AUC: 0.7344,AP: 0.6478
Epoch: 005, Loss: 4.3602,AUC: 0.7543,AP: 0.6685
Epoch: 006, Loss: 4.1259,AUC: 0.7735,AP: 0.6916
Epoch: 007, Loss: 3.8614,AUC: 0.7822,AP: 0.7011
Epoch: 008, Loss: 3.6699,AUC: 0.7904,AP: 0.7124
Epoch: 009, Loss: 3.4229,AUC: 0.7965,AP: 0.7244
Epoch: 010, Loss: 3.2225,AUC: 0.8001,AP: 0.7314
Epoch: 011, Loss: 2.9968,AUC: 0.8024,AP: 0.7365
Epoch: 012, Loss: 2.9058,AUC: 0.8050,AP: 0.7413
Epoch: 013, Loss: 2.6882,AUC: 0.8115,AP: 0.7568
Epoch: 014, Loss: 2.5923,AUC: 0.8174,AP: 0.7670
Epoch: 015, Loss: 2.4516,AUC: 0.8232,AP: 0.7759
Epoch: 016, Loss: 2.3360,AUC: 0.8281,AP: 0.7817
Epoch: 017, Loss: 2.2217,AUC: 0.8319,AP: 0.7872
Epoch: 018, Loss: 2.1104,AUC: 0.8371,AP: 0.7963
Epoch: 019, Loss: 2.0460,AUC: 0.8436,AP: 0.8072
Epoch: 020, Loss: 1.9642,AUC: 0.8494,

In [15]:
z5 = model()
z5.shape

torch.Size([898, 128])

In [16]:
zero_2 = torch.zeros((2, 128)).to(device)
z5 = torch.cat([z5, zero_2], 0)
z5.shape

torch.Size([900, 128])

In [36]:
# 将每个节点在不同快照学习到的特征拼接起来，使用最后一个快照进行测试
z_1_5 = torch.cat([z1, z2, z3, z4, z5], 1)
z_1_5.shape

torch.Size([900, 640])

In [37]:
# torch.save(z_1_5,'/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/z_1_5.pt')

In [39]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit

# z_1_5 = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/z_1_5.pt')
pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/fb_snapshots/m6.pt')
data = data.to(device)
data.num_nodes = 899

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.0,
                            is_undirected=True)
train_data, val_data, test_data = transform(data)

print(train_data.edge_index.shape)


@torch.no_grad()
def test():
    data = train_data
    z = z_1_5  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


AUC, AP = test()
print(f'AUC: {AUC:.4f},AP: {AP:.4f}')

torch.Size([2, 584])
AUC: 0.9570,AP: 0.9716
